In [148]:
from ase.io import read, write
import numpy as np
import pandas as pd
from pathlib import Path
from kmcos.run import KMC_Model
from kmcos.view import main
from kmcos.utils.make_superlattice_df import SuperlatticeDF
from ase import geometry 
from tqdm import tqdm
from kmcos.utils.surface_injection import InjectAtom

repeat = [20, 20, 20]
cell_ratio = 1
current = Path.cwd()
cif_path = current.parent / "Ru.cif"

def get_kmc_original_atoms(repeat):
    with KMC_Model(print_rates=False, banner=False) as model:
        model.settings.simulation_size = repeat
        atoms = model.get_atoms()
        return atoms

def make_slab(num_slab_layers, atoms):
    z_height = geometry.get_layers(atoms=atoms, miller=(0,0,1))[1][num_slab_layers]
    slab = atoms[atoms.get_positions()[:, 2] < z_height]
    return slab

In [149]:
kmc_original_atoms = get_kmc_original_atoms(repeat)
slab = make_slab(num_slab_layers=6, atoms=kmc_original_atoms)

1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
300 
0.1 
-0.02 


In [150]:
make_df = SuperlatticeDF(kmc_original_atoms, target_atoms=slab, repeat=repeat)
merged = make_df.df_merge(fill_empty_with="empty")
inner_coord_table = merged[["x_inner", "y_inner", "z_inner", "inner_coord_number"]].drop_duplicates()

inject_atom = InjectAtom(slab, kmc_original_atoms, critical_angle=10)

In [151]:
attached = inject_atom.get_adatom_positions()
inject_atom.convert_coord(attached, repeat, inner_coord_table)

[-5.39688015 20.25327065 10.68262458]


[2, 8, 3, 1]

In [152]:
total_cell = len(kmc_original_atoms)
current_atoms = len(slab)
print("the number of cells in total : ", total_cell)
print("the number of atoms as of now : ", current_atoms)
print("the available cells : ", total_cell - current_atoms)
num_of_iter = range(int((total_cell - current_atoms) / 50))

the number of cells in total :  16000
the number of atoms as of now :  2400
the available cells :  13600


In [153]:
atoms_list = []
with KMC_Model(print_rates=False, banner=False) as model:
    model.settings.simulation_size = repeat
    converter = {"ru": model.proclist.ru, "empty": model.proclist.empty}
    for coord, type in zip(merged[["x_cell", "y_cell", "z_cell", "inner_coord_number"]].values, merged["types_y"]):
        model._put(coord.tolist(), converter[type])
    model._adjust_database()
    model.parameters.T = 300
    model.do_steps(100_000)
    for iter in tqdm(num_of_iter):
        for _ in range(10):
            attached = inject_atom.get_adatom_positions()
            coord = inject_atom.convert_coord(attached, repeat, inner_coord_table)
            model._put(coord, converter["ru"])
        model._adjust_database()
        model.do_steps(10_000)
        atoms = model.get_atoms()
        atoms_list.append(atoms)

for atoms in atoms_list:
    write('atoms_trajectory.xyz', atoms, append = True)

1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
300 
0.1 
-0.02 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
300 
0.1 
-0.02 


  0%|          | 0/272 [00:00<?, ?it/s]

[12.14298034 17.91635301 10.68262458]
[29.68284082  1.55792954 10.68262458]
[-16.19064045  34.27477648  10.68262458]
[ 6.74610019 27.26402356 10.68262458]
[24.28596067 15.57943537 10.68262458]
[ 8.09532022 10.90560009 10.68262458]
[33.73050094 27.26402356 10.68262458]
[16.19064045 29.6009412  10.68262458]
[41.82582116 17.91635301 10.68262458]
[ 6.74610019 41.2855294  10.68262458]


  0%|          | 1/272 [00:00<01:54,  2.37it/s]

[41.82582116  8.56868245 10.68262458]
[16.19064045 38.94861176 10.68262458]
[ 0.         24.92710593 10.68262458]
[ 0.          1.55792954 10.68262458]
[-4.04766011 31.93785884 10.68262458]
[24.28596067 38.94861176 10.68262458]
[43.1750412  15.57943537 10.68262458]
[24.28596067  6.23176482 10.68262458]
[29.68284082 20.25327065 10.68262458]
[28.33362079  3.89484718 10.68262458]


...

[25.63518071  3.89484718 10.68262458]
[-5.39688015 10.90560009 10.68262458]
[ 8.09532022  1.55792954 10.68262458]
[20.23830056 17.91635301 10.68262458]
[32.3812809  34.27477648 10.68262458]
[-4.04766011 36.61169412 10.68262458]
[-10.7937603   24.92710593  10.68262458]
[20.23830056  8.56868245 10.68262458]
[-10.7937603   24.92710593  10.68262458]
[ 9.44454026 27.26402356 10.68262458]


  4%|▍         | 12/272 [00:04<01:44,  2.50it/s]

[-12.14298034  31.93785884  10.68262458]
[-17.53986049  36.61169412  10.68262458]
[ 9.44454026 31.93785884 10.68262458]
[10.7937603  24.92710593 10.68262458]
[-2.69844007 38.94861176 10.68262458]
[ 4.04766011 27.26402356 10.68262458]
[16.19064045 43.62244704 10.68262458]
[28.33362079 41.2855294  10.68262458]
[10.7937603  15.57943537 10.68262458]
[33.73050094 13.24251773 10.68262458]


  4%|▍         | 12/272 [00:05<01:53,  2.29it/s]


KeyboardInterrupt: 

array([ 0.        ,  2.13652492,  4.27304983,  6.40957475,  8.54609966,
       10.68262458, 12.81914949, 14.95567441, 17.09219933, 19.22872424,
       21.36524916, 23.50177407, 25.63829899, 27.7748239 , 29.91134882,
       32.04787374, 34.18439865, 36.32092357, 38.45744848, 40.5939734 ,
       42.73049831, 44.86702323, 47.00354815, 49.14007306, 51.27659798,
       53.41312289, 55.54964781, 57.68617272, 59.82269764, 61.95922256,
       64.09574747, 66.23227239, 68.3687973 , 70.50532222, 72.64184713,
       74.77837205, 76.91489697, 79.05142188, 81.1879468 , 83.32447171])

In [167]:
np.(slab.get_positions()[:, 2] - geometry.get_layers(slab, miller=[0,0,1])[1][2], axis=1) < 0.1

AxisError: axis 1 is out of bounds for array of dimension 1

In [163]:
slab[]

TypeError: len() of unsized object

In [161]:
geometry.get_layers(slab, miller=[0,0,1])[1][2]

4.27304983139038

In [493]:
adatom_number = np.arange(0, 600, 1)
atoms_list = []
with KMC_Model(print_rates=False, banner=False) as model:
    model.settings.simulation_size = repeat
    converter = {"ru": model.proclist.ru, "empty": model.proclist.empty}
    for coord, type in zip(merged[["x_cell", "y_cell", "z_cell", "inner_coord_number"]].values, merged["types_y"]):
        model._put(coord.tolist(), converter[type])
    model._adjust_database()
    model.parameters.T = 1600
    model.do_steps(100_000)
    atom = model.get_atoms()

1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1600 
0.1 
-0.02 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1.00 
1600 
0.1 
-0.02 


In [8]:
inject_atom = InjectAtom(slab, kmc_original_atoms, critical_angle=10)
attached = inject_atom.get_adatom_positions()
inner_coord_table = merged[["x_inner", "y_inner", "z_inner", "inner_coord_number"]].drop_duplicates()
inject_atom.convert_coord(attached, repeat, inner_coord_table)


NameError: name 'InjectAtom' is not defined